In [16]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv,find_dotenv
from IPython.display import display_markdown


load_dotenv(find_dotenv(),override=True)

client = Groq()

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

In [4]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [5]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array of integers using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array of integers to be sorted
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result
```
Here's an explanation of how the code works:

1. The `merge_sort` function takes an array of integers as input and recursively splits it into two halves until each half has only one element.
2. The `merge` function takes two sorted arrays as input and merges them into a single sorted array.
3. The `merge` function uses a temporary array `result` to store the merged elements.
4. The `merge` function iterates through both input arrays, comparing elements and adding the smaller one to the `result` array.
5. Once one of the input arrays is empty, the remaining elements from the other array are appended to the `result` array.
6. The `merge_sort` function returns the fully sorted array by recursively calling itself on the two halves and then merging the results.

You can test this implementation using a sample array, like this:
```
arr = [5, 2, 8, 3, 1, 6, 4]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Let me know if you have any questions or if you'd like me to revise anything!

### Reflection step

In [6]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [7]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

In [8]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [9]:
display_markdown(critique, raw=True)


Your Python implementation of the Merge Sort algorithm looks great! I'll provide some critique and recommendations to further improve the code.

**Code Quality and Readability**

Your code is well-structured, and the comments are informative. The variable names are descriptive, making it easy to understand the logic. However, there are a few areas where the code could be improved for better readability:

1.  Consistent Indentation: You're using 4-space indentation, which is good. However, in the `merge` function, the `result.extend(left)` and `result.extend(right)` lines are not aligned with the previous `while` loop. Make sure to maintain consistent indentation throughout the code.
2.  Blank Lines: Adding blank lines between functions can improve code readability. It makes the code easier to scan and understand.

**Functionality and Performance**

1.  Error Handling: Your implementation assumes that the input array contains only integers. If the input array contains other types of elements, the code may fail. Consider adding error handling to ensure the input array contains only comparable elements (e.g., numbers or strings).
2.  Stability: The current implementation of the `merge` function is stable, which means it preserves the order of equal elements. This is a desirable property in sorting algorithms. However, if you're not concerned about stability, you could optimize the `merge` function by using a single loop instead of recursively calling `merge_sort`.
3.  Input Validation: The `merge_sort` function doesn't validate the input. Consider adding a check to ensure the input is a list or other iterable. If not, raise a `TypeError` with a descriptive message.

**Best Practices and Pythonic Code**

1.  Type Hints: You've included docstrings, which are great. Consider adding type hints for function parameters and return types to make the code more readable and self-documenting.
2.  List Comprehensions: In the `merge` function, you're using `result.append` in a loop. Consider using a list comprehension instead, which can be more efficient and Pythonic.

Here's an updated version of your code incorporating these suggestions:
```
def merge_sort(arr: list) -> list:
    """
    Sorts an array of integers using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array of integers to be sorted
    :return: The sorted array
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left: list, right: list) -> list:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while left and right:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left or right)  # Use the truthy value of left or right
    return result

# Example usage
arr = [5, 2, 8, 3, 1, 6, 4]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Overall, your implementation is correct, and these suggestions are meant to improve the code's readability, maintainability, and Pythonic style.

In [10]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)


### Generation Step (II)


In [11]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [12]:
display_markdown(essay, raw=True)

Thank you for the detailed critique and suggestions! I'm glad to hear that the implementation is correct, and I appreciate the feedback on how to improve the code.

Here's a revised version of the code that incorporates your suggestions:
```
def merge_sort(arr: list) -> list:
    """
    Sorts an array of integers using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array of integers to be sorted
    :return: The sorted array
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left: list, right: list) -> list:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = [x for x in (left, right) for y in x if y is not None]
    result.sort()
    return result


# Example usage
arr = [5, 2, 8, 3, 1, 6, 4]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
Changes:

1.  I added a check at the beginning of `merge_sort` to ensure the input is a list. If not, a `TypeError` is raised.
2.  I used type hints for function parameters and return types to make the code more readable and self-documenting.
3.  I reformatted the code to maintain consistent indentation and added blank lines between functions for better readability.
4.  I replaced the `while` loop in the `merge` function with a list comprehension, which is more Pythonic and potentially more efficient.
5.  I used the `sort` method to sort the merged list in the `merge` function, which is more concise and efficient than the original implementation.

Let me know if this revised version meets your expectations!

### Implementing a class

In [19]:
from agentic_patterns import ReflectionAgent

In [20]:
agent = ReflectionAgent(model="llama3-70b-8192")


In [21]:
generation_system_prompt = "You are a Python programmer tasked with generating high quality Python code"

reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"

user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [23]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=10,
    verbose=1

)


STEP 1/10

 

GENERATION

 Here is a Python implementation of the Merge Sort algorithm:

```python
def merge_sort(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    return merge(merge_sort(left_half), merge_sort(right_half))


def merge(left, right):
    merged = []
    left_index = 0
    right_index = 0

    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


# Example usage:
arr = [64, 34, 25, 12, 22, 11, 90]
sorted_arr = merge_sort(arr)
print("Sorted array:", sorted_arr)
```

This implementation uses a recursive approach to split the input array into smaller halves until each half contains onl

In [24]:
display_markdown(final_response, raw=True)

Here is a Python implementation of the Merge Sort algorithm:

```python
def merge_sort(arr):
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left_half = arr[:mid]
    right_half = arr[mid:]

    return merge(merge_sort(left_half), merge_sort(right_half))


def merge(left, right):
    merged = []
    left_index = 0
    right_index = 0

    while left_index < len(left) and right_index < len(right):
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    merged.extend(left[left_index:])
    merged.extend(right[right_index:])

    return merged


# Example usage:
arr = [64, 34, 25, 12, 22, 11, 90]
sorted_arr = merge_sort(arr)
print("Sorted array:", sorted_arr)
```

This implementation uses a recursive approach to split the input array into smaller halves until each half contains only one element (or no elements, for an empty array). Then, it uses a merge function to combine the sorted halves back together, resulting in a fully sorted array. The merge function compares elements from the two halves and adds the smaller one to the resulting array, ensuring that the output is sorted in ascending order.